# Récupération des fichiers .mat dans les dossier r2_D_gamma 

In [ ]:
import os
import scipy.io
import pandas as pd
import numpy as np

# Spécifier le chemin du dossier contenant les fichiers .mat
directory_path = 'D:/Loriane/Résultats laboratoire/2023-08-10 SPT 4D6 R-HT Jurkat RL-HT/Control/r2_D_gamma'
parent_directory = os.path.dirname(directory_path)
output_excel_path = os.path.join(parent_directory, 'combined_output.xlsx')

# Dictionnaire pour regrouper les données des fichiers
data_dict = {}

# Fonction pour extraire la partie nécessaire du nom de fichier
def extract_base_name(filename):
    # Séparer le nom du fichier par "_"
    parts = filename.split('_')
    # Prendre la première partie (ex : "puit1")
    prefix = parts[0]
    # Rechercher la partie contenant "cell" avant "w1Quad-TIRF"
    suffix = next(part for part in parts if 'cell' in part)
    # Retourner le nom combiné (par ex : "puit1_cell1")
    return f"{prefix}_{suffix}"

# Parcourir tous les fichiers dans le dossier
for filename in os.listdir(directory_path):
    if filename.endswith('t1.xml_r2_D_gamma.mat'):
        # Extraire la bonne partie du nom
        base_name = extract_base_name(filename)
        file_path = os.path.join(directory_path, filename)
        mat_data = scipy.io.loadmat(file_path)

        # Extraire les données
        D = mat_data['D'].T.flatten() if 'D' in mat_data else []
        gamma = mat_data['gamma'].T.flatten() if 'gamma' in mat_data else []

        # Vérifier la présence de Quad-TIRF488, Quad-TIRF642 ou Quad-TIRF561
        if 'Quad-TIRF488' in filename or 'Quad-TIRF561' in filename:
            key_D = 'D RANKL'
            key_gamma = 'gamma RANKL'
        elif 'Quad-TIRF642' in filename:
            key_D = 'D RANK'
            key_gamma = 'gamma RANK'

        if base_name not in data_dict:
            data_dict[base_name] = {}

        # Vérifier si D ou gamma est vide
        if len(D) == 0 and len(gamma) == 0:
            continue  # Ne rien faire si les deux sont vides

        # Ajouter les données au dictionnaire
        data_dict[base_name][key_D] = D
        data_dict[base_name][key_gamma] = gamma

# Créer un writer Excel pour écrire les différentes feuilles
with pd.ExcelWriter(output_excel_path) as writer:
    for base_name, data in data_dict.items():
        # Si data est vide, passer cette feuille
        if not data:
            continue

        # Déterminer la longueur maximale des colonnes
        max_length = max(len(data[key]) for key in data.keys())
        
        # Compléter les colonnes avec des NaN si leur longueur est inférieure à la longueur maximale
        for key in data.keys():
            if len(data[key]) < max_length:
                data[key] = np.pad(data[key], (0, max_length - len(data[key])), constant_values=np.nan)

        # Créer le DataFrame
        df = pd.DataFrame(data)

        # Limiter le nom de la feuille à 31 caractères
        sheet_name = base_name[:31]
        
        try:
            print(df.head())
            df.to_excel(writer, sheet_name=sheet_name, index=False)
        except Exception as e:
            print(f"Erreur lors de l'écriture de la feuille {sheet_name}: {e}")

print(f"Le fichier Excel a été sauvegardé sous {output_excel_path}.")



Analyses de D de RANK et RANKL par GMM en sans fixer de composante (faite pour 4D6 seul et Jurkat seul pour calculer les valeurs de la composante lente)

In [ ]:
#bon code pour les valeurs entre 10^-4 et 10^0 

import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy.integrate import simps  # Pour l'intégration numérique
import os

# Charger le fichier Excel
file_path = '/Users/lorie/Library/CloudStorage/GoogleDrive-maillot.loriane@gmail.com/Mon Drive/thèse/Résultats/microscopie/Résultats laboratoire/Code matalab et python Loriane/Gaussian/dossier relance gamma D traj long/combined_output_2024-07-09_Jurkatseul.xlsx'
excel_file = pd.ExcelFile(file_path)

# Colonnes que nous allons traiter
columns = ['Diff RANK', 'Diff RANKL']

# Paramètres pour les calculs
pixel = 0.11  # en micromètres
#trackmate_time = 0.248  # en secondes si 1 couleur : 0.124, si 2 couleurs : 0.248, si 4 couleurs : 0.43
time_lag = 0.248 #/ trackmate_time  # en secondes

# Parcourir chaque feuille du fichier Excel
for sheet_name in excel_file.sheet_names:
    data = pd.read_excel(file_path, sheet_name=sheet_name)
    
    # Calculer Diff RANK et Diff RANKL
    if 'D RANK' in data.columns:
        data['Diff RANK'] = data['D RANK'] * (pixel ** 2) / time_lag
    if 'D RANKL' in data.columns:
        data['Diff RANKL'] = data['D RANKL'] * (pixel ** 2) / time_lag

    # Générer le nom du fichier de sortie pour les paramètres
    output_file = f'gmm_parameters_{os.path.basename(file_path)}_{sheet_name}.txt'
    
    with open(output_file, 'w') as f:
        f.write(f"Analyse des composantes GMM pour le fichier : {file_path}, feuille : {sheet_name}\n")
        f.write("=" * 50 + "\n\n")

        # Fonction pour appliquer le GMM et tracer les graphes pour les populations fast et slow
        def analyze_and_plot(column_name):
            column_data = data[column_name].dropna()  # Supprimer les valeurs manquantes

            # Appliquer la transformation logarithmique (log10)
            log_data = np.log10(column_data[column_data > 0])  # Exclure les valeurs <= 0

            # Limiter les données à l'intervalle [-4, 0] (car 10^-4 à 10^0 correspond à cet intervalle sur log10)
            log_data = log_data[(log_data >= -4) & (log_data <= 0)]

            # Reshape des données pour le GMM
            log_data_reshaped = log_data.values.reshape(-1, 1)

            # Appliquer le modèle GMM avec deux composantes (fast et slow)
            gmm = GaussianMixture(n_components=2, covariance_type='full')
            gmm.fit(log_data_reshaped)

            # Obtenir les paramètres du GMM
            means = gmm.means_.flatten()
            covariances = np.sqrt(gmm.covariances_).flatten()  # Extraire les écarts-types
            weights = gmm.weights_

            # Réordonner les composantes pour garantir que la lente (slow) est toujours avant la rapide (fast)
            if means[0] > means[1]:
                means = means[::-1]  # Inverser l'ordre des moyennes
                covariances = covariances[::-1]  # Inverser les écarts-types
                weights = weights[::-1]  # Inverser les poids

            # Calculer le seuil qui sépare fast et slow (moyenne des deux moyennes)
            threshold = (means[0] + means[1]) / 2

            # Prédire les labels (fast/slow) pour chaque point de données en fonction des moyennes réordonnées
            labels = np.where(log_data < threshold, 0, 1)  # 0 pour slow, 1 pour fast

            # Calculer le pourcentage de chaque sous-population
            slow_percentage = np.sum(labels == 0) / len(labels) * 100
            fast_percentage = np.sum(labels == 1) / len(labels) * 100

            # Enregistrer les paramètres dans le fichier texte
            f.write(f"Colonne analysée : {column_name}\n")
            f.write(f"Pourcentage de molécules lentes (slow) : {slow_percentage:.2f}%\n")
            f.write(f"Pourcentage de molécules rapides (fast) : {fast_percentage:.2f}%\n")
            f.write(f"Moyennes des composantes GMM : {means}\n")
            f.write(f"Écarts-types des composantes GMM : {covariances}\n")
            f.write(f"Seuil de séparation fast/slow : {threshold}\n")

            # Tracer un histogramme des données avec les deux composantes du GMM
            sns.histplot(log_data, bins=30, kde=False, color='gray', label=f'Données {column_name}', stat='density')

            # Tracer la densité de probabilité pour chaque composante du GMM
            x_vals = np.linspace(-4, 0, 1000)  # Limiter l'intervalle de traçage entre -4 et 0

            # Calculer la densité pour chaque composante
            pdf_0 = norm.pdf(x_vals, means[0], covariances[0])
            pdf_1 = norm.pdf(x_vals, means[1], covariances[1])

            # Plot the individual component PDFs, scaled by the GMM weights
            plt.plot(x_vals, weights[0] * pdf_0, label='Composante Lente (Slow)', color='green')
            plt.plot(x_vals, weights[1] * pdf_1, label='Composante Rapide (Fast)', color='orange')

            # Tracer la densité totale du GMM
            pdf_total = weights[0] * pdf_0 + weights[1] * pdf_1  # Densité totale comme somme pondérée
            plt.plot(x_vals, pdf_total, label='Ajustement GMM', color='red')

            # Marquer le seuil de séparation
            plt.axvline(x=threshold, color='blue', linestyle='--', label='Seuil Fast/Slow')

            # Ajouter des légendes et titres
            plt.legend()
            plt.xlabel('log10(Données)')
            plt.ylabel('Fréquence')
            plt.title(f'Modèle GMM pour {column_name} (Fast vs Slow) - Feuille {sheet_name}')

            # Limiter les axes à l'intervalle [-4, 0]
            plt.xlim([-4, 0])

            # Sauvegarder la figure avec nom complet
            figure_name = f'gmm_{column_name}_{os.path.basename(file_path)}_{sheet_name}.png'
            plt.savefig(figure_name)
            plt.close()  # Fermer la figure pour ne pas l'afficher dans l'environnement actuel

            # Calcul de l'air sous la courbe pour chaque composante et l'ajustement total
            area_slow = simps(weights[0] * pdf_0, x_vals)
            area_fast = simps(weights[1] * pdf_1, x_vals)
            area_total = simps(pdf_total, x_vals)

            # Enregistrer les aires sous la courbe dans le fichier texte
            f.write(f"Aire sous la courbe de la composante Lente (Slow) : {area_slow:.4f}\n")
            f.write(f"Aire sous la courbe de la composante Rapide (Fast) : {area_fast:.4f}\n")
            f.write(f"Aire sous la courbe de l'ajustement GMM total : {area_total:.4f}\n")
            f.write("=" * 50 + "\n\n")

        # Appliquer l'analyse uniquement sur 'Diff RANK' et 'Diff RANKL'
        for col in ['Diff RANK', 'Diff RANKL']:
            if col in data.columns:
                analyze_and_plot(col)

print(f"Les résultats ont été sauvegardés et les figures ont été enregistrées.")


Analyses de D de RANK et RANKL par GMM en fixant la fraction lente uniquement 

In [ ]:
#code bon pour fixer composante lente uniquement 
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy.integrate import simps  # Pour l'intégration numérique
from sklearn.metrics import r2_score
import os

# Charger le fichier Excel
file_path = '/Users/lorie/Library/CloudStorage/GoogleDrive-maillot.loriane@gmail.com/Mon Drive/thèse/Résultats/microscopie/Résultats laboratoire/Code matalab et python Loriane/Gaussian/output Trackmate Jurkat Control Vs Ac/combined_output Control 29 02 2024.xlsx'
excel_file = pd.ExcelFile(file_path)

# Colonnes que nous allons traiter
columns = ['Diff RANK', 'Diff RANKL']

# Paramètres pour les calculs
pixel = 0.11  # en micromètres
#trackmate_time = 0.248  # en secondes si 1 couleur : 0.124, si 2 couleurs : 0.248, si 4 couleurs : 0.43
time_lag = 0.248  # en secondes

# Parcourir chaque feuille du fichier Excel
for sheet_name in excel_file.sheet_names:
    data = pd.read_excel(file_path, sheet_name=sheet_name)
    
    # Calculer Diff RANK et Diff RANKL
    if 'D RANK' in data.columns:
        data['Diff RANK'] = data['D RANK'] * (pixel ** 2) / time_lag
    if 'D RANKL' in data.columns:
        data['Diff RANKL'] = data['D RANKL'] * (pixel ** 2) / time_lag

    # Générer le nom du fichier de sortie pour les paramètres
    output_file = f'gmm_parameters_{os.path.basename(file_path)}_{sheet_name}.txt'
    
    with open(output_file, 'w') as f:
        f.write(f"Analyse des composantes GMM pour le fichier : {file_path}, feuille : {sheet_name}\n")
        f.write("=" * 50 + "\n\n")

        # Fonction pour appliquer le GMM et tracer les graphes pour les populations fast et slow
        def analyze_and_plot(column_name):
            column_data = data[column_name].dropna()  # Supprimer les valeurs manquantes

            # Appliquer la transformation logarithmique (log10)
            log_data = np.log10(column_data[column_data > 0])  # Exclure les valeurs <= 0

            # Limiter les données à l'intervalle [-4, 0] (car 10^-4 à 10^0 correspond à cet intervalle sur log10)
            log_data = log_data[(log_data >= -4) & (log_data <= 0)]

            # Reshape des données pour le GMM
            log_data_reshaped = log_data.values.reshape(-1, 1)

            # Fixer uniquement la première composante pour Diff RANK ou Diff RANKL
            if column_name == 'Diff RANK':
                fixed_means = np.array([[-2.77749681]])  # Moyenne de la composante lente pour RANK
                fixed_covariances = np.array([[[0.492384**2]]])  # Variance de la composante lente pour RANK
            elif column_name == 'Diff RANKL':
                fixed_means = np.array([[-2.648777307]])  # Moyenne de la composante lente pour RANKL
                fixed_covariances = np.array([[[0.455597573**2]]])  # Variance de la composante lente pour RANKL
            else:
                return

            # Initialiser le modèle GMM
            gmm = GaussianMixture(n_components=2, covariance_type='full')

            # Fixer les paramètres de la première composante (lente)
            gmm.means_init = np.vstack([fixed_means, np.mean(log_data_reshaped, axis=0)])  # Moyenne de la deuxième composante à estimer
            gmm.fit(log_data_reshaped)

            # Pour garantir que la première composante reste fixée, on remplace après l'ajustement
            gmm.means_[0] = fixed_means
            gmm.covariances_[0] = fixed_covariances
            gmm.precisions_cholesky_ = np.linalg.cholesky(np.linalg.inv(gmm.covariances_))

            # Extraire les moyennes et covariances finales
            means = gmm.means_.flatten()
            covariances = np.sqrt(np.array([np.linalg.inv(gmm.precisions_[i]).flatten()[0] for i in range(2)]))
            weights = gmm.weights_

            # Calculer le seuil de séparation fast/slow
            threshold = (means[0] + means[1]) / 2

            # Prédire les labels pour chaque point
            labels = np.where(log_data < threshold, 0, 1)

            # Calculer les pourcentages
            slow_percentage = np.sum(labels == 0) / len(labels) * 100
            fast_percentage = np.sum(labels == 1) / len(labels) * 100

            # Enregistrer les paramètres
            f.write(f"Colonne analysée : {column_name}\n")
            f.write(f"Pourcentage de molécules lentes (slow) : {slow_percentage:.2f}%\n")
            f.write(f"Pourcentage de molécules rapides (fast) : {fast_percentage:.2f}%\n")
            f.write(f"Moyennes des composantes GMM : {means}\n")
            f.write(f"Écarts-types des composantes GMM : {covariances}\n")
            f.write(f"Seuil de séparation fast/slow : {threshold}\n")

            # Tracer les histogrammes et les densités
            sns.histplot(log_data, bins=30, kde=False, color='gray', label=f'Données {column_name}', stat='density')

            # Calculer les densités de probabilité pour les composantes
            x_vals = np.linspace(-4, 0, 1000)

            # Densités pour les composantes lente et rapide
            pdf_0 = norm.pdf(x_vals, means[0], covariances[0])
            pdf_1 = norm.pdf(x_vals, means[1], covariances[1])

            # Tracer les composantes pondérées
            plt.plot(x_vals, weights[0] * pdf_0, label='Composante Lente (Slow)', color='green')
            plt.plot(x_vals, weights[1] * pdf_1, label='Composante Rapide (Fast)', color='orange')

            # Densité totale du GMM
            pdf_total = weights[0] * pdf_0 + weights[1] * pdf_1
            plt.plot(x_vals, pdf_total, label='Ajustement GMM', color='red')

            # Calcul de l'aire sous la courbe
            area_slow = simps(weights[0] * pdf_0, x_vals)
            area_fast = simps(weights[1] * pdf_1, x_vals)
            area_total = simps(pdf_total, x_vals)

            # Calcul du R² pour mesurer l'ajustement
            r2 = r2_score(log_data, np.interp(log_data, x_vals, pdf_total))

            # Enregistrer l'ajustement et R² dans le fichier texte
            f.write(f"Aire sous la courbe Slow : {area_slow:.4f}\n")
            f.write(f"Aire sous la courbe Fast : {area_fast:.4f}\n")
            f.write(f"Aire totale : {area_total:.4f}\n")
            f.write(f"R² de l'ajustement : {r2:.4f}\n")
            f.write("=" * 50 + "\n\n")

            # Ajouter le seuil de séparation sur le graphique
            plt.axvline(x=threshold, color='blue', linestyle='--', label='Seuil Fast/Slow')
            plt.legend()

            # Sauvegarder la figure
            figure_name = f'gmm_{column_name}_{os.path.basename(file_path)}_{sheet_name}.png'
            plt.savefig(figure_name)
            plt.close()

        # Appliquer l'analyse pour les colonnes RANK et RANKL
        for col in ['Diff RANK', 'Diff RANKL']:
            if col in data.columns:
                analyze_and_plot(col)

print(f"Les résultats ont été sauvegardés et les figures enregistrées.")


Code qui permet de récupérer l'ensemble des fichiers textes pour faire un tableau comportant les pourcentages 

In [ ]:
#bon code 
import pandas as pd
import os

# Fonction pour traiter un fichier texte GMM
def process_gmm_txt(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Extraire les données pour D RANKL et D RANK
    data = {}
    current_col = None
    for line in lines:
        if "Colonne analysée" in line:
            current_col = line.strip().split(':')[-1].strip()
            data[current_col] = {}
        elif "Pourcentage de molécules lentes" in line:
            data[current_col]["Pourcentage de molécules lentes (slow)"] = line.split(':')[-1].strip()
        elif "Pourcentage de molécules rapides" in line:
            data[current_col]["Pourcentage de molécules rapides (fast)"] = line.split(':')[-1].strip()
        elif "Moyennes des composantes GMM" in line:
            data[current_col]["Moyennes des composantes GMM"] = line.split(':')[-1].strip()
        elif "Écarts-types des composantes GMM" in line:
            data[current_col]["Écarts-types des composantes GMM"] = line.split(':')[-1].strip()
        elif "Seuil de séparation fast/slow" in line:
            data[current_col]["Seuil de séparation fast/slow"] = line.split(':')[-1].strip()
        elif "Aire sous la courbe de la composante Lente" in line:
            data[current_col]["Aire sous la courbe de la composante Lente (Slow)"] = line.split(':')[-1].strip()
        elif "Aire sous la courbe de la composante Rapide" in line:
            data[current_col]["Aire sous la courbe de la composante Rapide (Fast)"] = line.split(':')[-1].strip()
        elif "Aire sous la courbe de l'ajustement GMM total" in line:
            data[current_col]["Aire sous la courbe de l'ajustement GMM total"] = line.split(':')[-1].strip()
    
    return data

# Fonction principale pour traiter tous les fichiers texte dans un dossier
def process_all_gmm_files_in_folder(folder_path):
    # Créer des listes pour stocker les données pour D RANKL et D RANK
    rankl_data = []
    rank_data = []

    # Colonnes pour le fichier Excel
    columns = [
        "Nom du fichier", 
        "Pourcentage de molécules lentes (slow)", 
        "Pourcentage de molécules rapides (fast)", 
        "Moyennes des composantes GMM", 
        "Écarts-types des composantes GMM", 
        "Seuil de séparation fast/slow", 
        "Aire sous la courbe de la composante Lente (Slow)", 
        "Aire sous la courbe de la composante Rapide (Fast)", 
        "Aire sous la courbe de l'ajustement GMM total"
    ]

    # Parcourir tous les fichiers .txt dans le dossier
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(folder_path, filename)
            
            # Traiter le fichier texte et extraire les données
            gmm_data = process_gmm_txt(file_path)
            
            # Ajouter les données pour D RANKL et D RANK dans leurs listes respectives
            if "Diff RANKL" in gmm_data:
                rankl_data.append({
                    "Nom du fichier": filename,
                    **gmm_data["Diff RANKL"]
                })
            if "Diff RANK" in gmm_data:
                rank_data.append({
                    "Nom du fichier": filename,
                    **gmm_data["Diff RANK"]
                })

    # Créer des DataFrames pour D RANKL et D RANK
    df_rankl = pd.DataFrame(rankl_data, columns=columns)
    df_rank = pd.DataFrame(rank_data, columns=columns)

    # Enregistrer les données dans un fichier Excel avec deux feuilles
    output_excel_path = os.path.join(folder_path, 'gmm_combined_output.xlsx')
    with pd.ExcelWriter(output_excel_path) as writer:
        df_rankl.to_excel(writer, sheet_name='Diff RANKL', index=False)
        df_rank.to_excel(writer, sheet_name='Diff RANK', index=False)
    
    print(f"Le fichier Excel a été enregistré sous : {output_excel_path}")

# Spécifier le dossier contenant les fichiers .txt
folder_path = '/Users/lorie/Library/CloudStorage/GoogleDrive-maillot.loriane@gmail.com/Mon Drive/thèse/Résultats/microscopie/Résultats laboratoire/Code matalab et python Loriane/Gaussian/dossier test longtrajgamma'  # Remplacez par le chemin du dossier où sont stockés vos fichiers .txt

# Exécuter la fonction pour traiter tous les fichiers dans le dossier
process_all_gmm_files_in_folder(folder_path)


Récupérer les gamma dans combined_output_202* avec valeurs fixe des seuils calcul des % des différentes populations 

In [ ]:
# pour plusieurs dossiers excels 
import pandas as pd
import numpy as np
import os

# Définir les seuils
CONFINE_LIMIT = 0.75
BROWNIAN_LIMIT = 1.25

# Fonction pour analyser les données et générer les fichiers Excel consolidés
def analyze_multiple_excel_files(file_paths, output_folder):
    # Initialiser des dictionnaires pour stocker les DataFrames par fichier d'entrée
    rank_sheets = {}
    rankl_sheets = {}

    for file_path in file_paths:
        # Charger le fichier Excel
        excel_data = pd.ExcelFile(file_path)
        rank_results = []
        rankl_results = []

        # Parcourir chaque feuille
        for sheet_name in excel_data.sheet_names:
            df = excel_data.parse(sheet_name)

            for column, results in [("gamma RANK", rank_results), ("gamma RANKL", rankl_results)]:
                if column in df.columns:
                    # Extraire et filtrer les valeurs
                    gamma_values = df[column].dropna()
                    #gamma_values = gamma_values[(gamma_values > 0) & (gamma_values < 3)]
                    gamma_values = gamma_values[gamma_values != 0] # exlu uniquement les 0

                    if len(gamma_values) == 0:
                        continue  # Passer si aucune valeur valide

                    # Calculer les pourcentages
                    total_values = len(gamma_values)
                    confined_count = len(gamma_values[gamma_values < CONFINE_LIMIT])
                    brownian_count = len(gamma_values[(gamma_values >= CONFINE_LIMIT) & (gamma_values < BROWNIAN_LIMIT)])
                    linear_count = len(gamma_values[gamma_values >= BROWNIAN_LIMIT])

                    confined_percent = (confined_count / total_values) * 100
                    brownian_percent = (brownian_count / total_values) * 100
                    linear_percent = (linear_count / total_values) * 100

                    # Ajouter les résultats à la liste
                    results.append({
                        "Nom du fichier": os.path.basename(file_path),
                        "Nom de la feuille": sheet_name,
                        "% Confiné": confined_percent,
                        "% Brownien": brownian_percent,
                        "% Linéaire": linear_percent
                    })

        # Ajouter les résultats pour ce fichier à un DataFrame
        rank_sheets[os.path.basename(file_path)] = pd.DataFrame(rank_results)
        rankl_sheets[os.path.basename(file_path)] = pd.DataFrame(rankl_results)

    # Créer des fichiers Excel consolidés avec plusieurs feuilles
    rank_output_file = os.path.join(output_folder, "gamma_RANK_percentages.xlsx")
    rankl_output_file = os.path.join(output_folder, "gamma_RANKL_percentages.xlsx")

    with pd.ExcelWriter(rank_output_file) as writer:
        for file_name, df in rank_sheets.items():
            df.to_excel(writer, sheet_name=file_name[:31], index=False)  # Limiter le nom de la feuille à 31 caractères

    with pd.ExcelWriter(rankl_output_file) as writer:
        for file_name, df in rankl_sheets.items():
            df.to_excel(writer, sheet_name=file_name[:31], index=False)

    print(f"Fichiers Excel consolidés générés : {rank_output_file}, {rankl_output_file}")

# Liste des fichiers Excel à analyser
file_paths = [
    '/Users/lorie/Library/CloudStorage/GoogleDrive-maillot.loriane@gmail.com/Mon Drive/thèse/Résultats/microscopie/Résultats laboratoire/Code matalab et python Loriane/Gaussian/dossier relance gamma D traj long/+Ac/combined_output_2023-08-10_+Ac.xlsx',
    '/Users/lorie/Library/CloudStorage/GoogleDrive-maillot.loriane@gmail.com/Mon Drive/thèse/Résultats/microscopie/Résultats laboratoire/Code matalab et python Loriane/Gaussian/dossier relance gamma D traj long/+Ac/combined_output_2023-08-10_Control.xlsx',
    '/Users/lorie/Library/CloudStorage/GoogleDrive-maillot.loriane@gmail.com/Mon Drive/thèse/Résultats/microscopie/Résultats laboratoire/Code matalab et python Loriane/Gaussian/dossier relance gamma D traj long/+Ac/combined_output_2024-02-13_+Ac.xlsx',
    '/Users/lorie/Library/CloudStorage/GoogleDrive-maillot.loriane@gmail.com/Mon Drive/thèse/Résultats/microscopie/Résultats laboratoire/Code matalab et python Loriane/Gaussian/dossier relance gamma D traj long/+Ac/combined_output_2024-02-13_Control.xlsx',
    '/Users/lorie/Library/CloudStorage/GoogleDrive-maillot.loriane@gmail.com/Mon Drive/thèse/Résultats/microscopie/Résultats laboratoire/Code matalab et python Loriane/Gaussian/dossier relance gamma D traj long/+Ac/combined_output_2024-02-14_+Ac.xlsx',
    '/Users/lorie/Library/CloudStorage/GoogleDrive-maillot.loriane@gmail.com/Mon Drive/thèse/Résultats/microscopie/Résultats laboratoire/Code matalab et python Loriane/Gaussian/dossier relance gamma D traj long/+Ac/combined_output_2024-02-14_Control.xlsx',
    '/Users/lorie/Library/CloudStorage/GoogleDrive-maillot.loriane@gmail.com/Mon Drive/thèse/Résultats/microscopie/Résultats laboratoire/Code matalab et python Loriane/Gaussian/dossier relance gamma D traj long/+Ac/combined_output_2024-02-29_+Ac.xlsx',
    '/Users/lorie/Library/CloudStorage/GoogleDrive-maillot.loriane@gmail.com/Mon Drive/thèse/Résultats/microscopie/Résultats laboratoire/Code matalab et python Loriane/Gaussian/dossier relance gamma D traj long/+Ac/combined_output_2024-02-29_Control.xlsx',
 ]

# Dossier de sortie
output_folder = '/Users/lorie/Library/CloudStorage/GoogleDrive-maillot.loriane@gmail.com/Mon Drive/thèse/Résultats/microscopie/Résultats laboratoire/Code matalab et python Loriane/Gaussian/dossier relance gamma D traj long/+Ac'

# Exécuter la fonction
analyze_multiple_excel_files(file_paths, output_folder)